In [1]:
import requests
from requests.auth import HTTPBasicAuth
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

In [2]:
# Vito service username and password. we call this information from config.py file.
import config
username=config.username
password=config.password

In [6]:
url='https://land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Properties/LAI_1km_V2/'

In [7]:
response = requests.get(url,auth=(username,password))
soup=BeautifulSoup(response.text,'html.parser')
years_divs= soup.find_all("td",{"class": "bydate"})

In [9]:
folder='../geoserver_data/2020_Cop_LAI/'

In [ ]:
for y in years_divs[1:]:
    url_year=y.a['href']
    
    ###########################################
    # we use if blog for downloading spesific year
    #if int(url_year[-5:-1])>2018:
        #continue
    #if int(url_year[-5:-1])<2000:
        #continue
    ###########################################
    
    download_path=folder+str(url_year[-5:-1])
    try:
        os.mkdir(download_path)
    except FileExistsError:
        pass
    
    response_year = requests.get(url_year,auth=(username,password))
    page_year=BeautifulSoup(response_year.text,'html.parser')
    mounth_divs = page_year.find_all("td",{"class": "bydate"})
    
    if len(mounth_divs)==0:
        print(str(url_year)+' year is empty')
        with open(download_path+'/year_nodata_list', 'a') as t:
                    t.write("%s\n" % str(url_year))
        continue
    time.sleep(1)
    for m in mounth_divs[1:]:
        url_mounth=m.a['href']
        response_mounth = requests.get(url_mounth,auth=(username,password))
        page_mounth=BeautifulSoup(response_mounth.text,'html.parser')
        days_divs = page_mounth.find_all("td",{"class": "bydate"})
        if len(days_divs)==0:
            print(str(url_mounth)+' mounth is empty')
            with open(download_path+'/mounth_nodata_list', 'a') as t:
                    t.write("%s\n" % str(url_mounth))
            
            continue
        time.sleep(1)
        for d in days_divs[1:]:
            url_day=d.a['href']
            response_day_v2 = requests.get(url_day,auth=(username,password))
            page_day_v2=BeautifulSoup(response_day_v2.text,'html.parser')
            days_divs_v2 = page_day_v2.find_all("td",{"class": "bydate"})
            if len(days_divs_v2)==0:
                print(str(url_day)+' is empty')
                with open(download_path+'/daily_nodata_list', 'a') as t:
                    t.write("%s\n" % str(url_day))                
                continue
                
            url_day_v2=days_divs_v2[1].a['href']
            response_img = requests.get(url_day_v2,auth=(username,password))
            page_img=BeautifulSoup(response_img.text,'html.parser')
            img_divs = page_img.find_all('td')
            tag=img_divs[2]
            nc_name=tag.text[6:-15]
            nc_url=url_day_v2+nc_name
            image_bool=False
            try:
                while image_bool is False:
                    try:
                        #print(nc_url)
                        response = requests.get(nc_url, stream=True,auth=(username,password))
                        if response.status_code == 200:
                            with open(download_path+'/'+nc_name, 'wb') as f:
                                f.write(response.raw.read())
                                image_bool=True

                        time.sleep(4)
                    
                    except ProtocolError:
                        time.sleep(5)
                        image_bool=False
                        pass
                    except ConnectionError:
                        time.sleep(5)
                        image_bool=False
                        pass
                    except:
                        time.sleep(5)
                        image_bool=False
                        pass
                        
                
                with open(download_path+'/download_list', 'a') as t:
                        t.write("%s\n" % nc_url)                                          
                    
                
                    
            except Exception as e:
                with open(download_path+'/error_list', 'a') as t:
                    t.write("%s\n" % nc_url)
                
                continue
                
        
            
        time.sleep(10)   
            
            
        
    time.sleep(30)
    print(download_path)    
    print('-------------------------------------------')

